In [5]:
!pip install -q transformers datasets accelerate torch faiss-cpu sentence-transformers PyPDF2 streamlit pyngrok
!npm install localtunnel


⠙⠹⠸⠼⠴
up to date, audited 23 packages in 889ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

In [6]:
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments

# 1. Load Dataset (SQuAD - Stanford Question Answering Dataset)
print("Loading dataset...")
dataset = load_dataset("squad", split="train[:500]")  # Using small subset for speed
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# 2. Preprocessing
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def prepare_train_features(examples):
    # Tokenize questions and contexts
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

tokenized_datasets = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

# 3. Define Model
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

# 4. Training Arguments
args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,  # Increase epochs if needed
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',  # Optional: helps with logging
    logging_steps=500,     # Optional: number of steps to log
    save_steps=500,
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

print("Starting Training...")
trainer.train()
print("Training Complete! Model Saved.")

# Save the fine-tuned model
model.save_pretrained("./my_fine_tuned_distilbert")
tokenizer.save_pretrained("./my_fine_tuned_distilbert")


Loading dataset...


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2191131199.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting Training...


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss


Training Complete! Model Saved.


('./my_fine_tuned_distilbert/tokenizer_config.json',
 './my_fine_tuned_distilbert/special_tokens_map.json',
 './my_fine_tuned_distilbert/vocab.txt',
 './my_fine_tuned_distilbert/added_tokens.json',
 './my_fine_tuned_distilbert/tokenizer.json')

In [7]:
!apt-get install -y libfaiss-dev
!pip install faiss-cpu
!apt-get install -y libfaiss-dev
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from PyPDF2 import PdfReader

# 1. Initialize Components
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # Sentence Embedding Model
qa_pipeline = pipeline("question-answering", model="./my_fine_tuned_distilbert", tokenizer="./my_fine_tuned_distilbert")

def process_pdf(pdf_file):
    """Extract text from PDF and chunk it."""
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    # Simple chunking by splitting on newlines or length
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    return chunks

def create_faiss_index(chunks):
    """Create FAISS index from text chunks."""
    embeddings = embedder.encode(chunks)
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(np.array(embeddings))
    return index, chunks

def get_answer(question, index, chunks):
    """Retrieve context and generate answer."""
    # 1. Retrieve
    q_embedding = embedder.encode([question])
    D, I = index.search(np.array(q_embedding), k=5)  # Get top 3 relevant chunks

    context = " ".join([chunks[i] for i in I[0]])

    # 2. Read (Answer Extraction)
    result = qa_pipeline(question=question, context=context)
    return result['answer'], context


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libfaiss-dev
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 949 kB of archives.
After this operation, 6,224 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libfaiss-dev amd64 1.7.2-5 [949 kB]
Fetched 949 kB in 2s (607 kB/s)
Selecting previously unselected package libfaiss-dev:amd64.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../libfaiss-dev_1.7.2-5_amd64.deb ...
Unpacking libfaiss-dev:amd64 (1.7.2-5) ...
Setting up libfaiss-dev:amd64 (1.7.2-5) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libfaiss-dev is already the newest version (1.7.2-5).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
%%writefile app.py
import streamlit as st
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from PyPDF2 import PdfReader
import os

# --- Configuration ---
st.set_page_config(page_title="Document Q&A System", layout="centered")

# --- Load Models (Cached) ---
@st.cache_resource
def load_models():
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad", tokenizer="bert-large-uncased")

    return embedder, qa_pipeline

embedder, qa_pipeline = load_models()

# --- Helpers ---
def process_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    # Use larger chunks
    chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]  # Bigger chunks
    return chunks

# --- UI ---
st.markdown("<h1 style='text-align: center; color: #2c3e50;'>🧠 Document Q&A System (RAG)</h1>", unsafe_allow_html=True)

if 'index' not in st.session_state:
    st.session_state.index = None
if 'chunks' not in st.session_state:
    st.session_state.chunks = []

with st.container():
    st.markdown("### 📄 Document Source")
    uploaded_file = st.file_uploader("Upload your PDF", type=['pdf'])

    if uploaded_file and not st.session_state.index:
        with st.spinner("Processing & Indexing..."):
            chunks = process_pdf(uploaded_file)
            embeddings = embedder.encode(chunks)
            index = faiss.IndexFlatL2(embeddings.shape[1])
            index.add(np.array(embeddings))

            st.session_state.index = index
            st.session_state.chunks = chunks
            st.success("Document Indexed Successfully!")

with st.container():
    st.markdown("### 💬 Q&A Interface")
    query = st.chat_input("Ask a question about the document...")

    if query:
        with st.chat_message("user"):
            st.write(query)

        if st.session_state.index:
            # RAG Retrieval
            q_embed = embedder.encode([query])
            D, I = st.session_state.index.search(np.array(q_embed), k=3)
            context = " ".join([st.session_state.chunks[i] for i in I[0]])

            # AI Answer
            result = qa_pipeline(question=query, context=context)

            with st.chat_message("assistant"):
                st.write(result['answer'])
                with st.expander("View Context"):
                    st.write(context)
        else:
            st.error("Please upload a document first.")


Writing app.py


In [9]:
from pyngrok import ngrok
import os

# Set your ngrok authtoken (replace 'YOUR_AUTH_TOKEN' with the token you copied)
ngrok.set_auth_token('36lgZcNFuhX77t8AFgepxgzI0pl_2ZDjRc7oVpt9toqQLEWWj')

# Set up the ngrok tunnel (Streamlit runs on port 8501 by default)
public_url = ngrok.connect(8501)

# Run the Streamlit app in the background
os.system('streamlit run app.py &')

# Display the public URL to access the app
print('Streamlit is live at:', public_url)

Streamlit is live at: NgrokTunnel: "https://laurette-improper-ronnie.ngrok-free.dev" -> "http://localhost:8501"
